Data Acquisition

In [1]:
# Realizamos la importaciones que necesitaremos a lo largo del proyecto

import numpy as np
import pandas as pd
import re
import seaborn as sns
import argparse


In [2]:
# Importamos el archivo csv

dfApp = pd.read_csv('googleplaystore.csv', encoding = "utf-8")
dfReview = pd.read_csv('googleplaystore_user_reviews.csv', encoding = "utf-8")
dfApp.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [3]:
dfApp.dtypes
len(dfApp)

10841

DATA CLEANING

In [4]:

dfApp = dfApp.drop_duplicates()
len(dfApp)

10358

In [ ]:
# No es necesario manipular 'rating'
# Convertmos a MB todas las medidas de 'Size'.

def toMb(x):
    if 'M' in str(x):
        return str(x).replace('M', '')
    elif 'k' in str(x):
        return float(str(x).replace('k', '')) / 1000
    else:
        return str(x).replace('Varies with device', 'NaN')
 



dfApp['Size'] = dfApp['Size'].apply(lambda x: toMb(x))
dfApp.loc[(dfApp['Size']== '1,000+', 'Size')] = str(1)
dfApp['Size'] = dfApp['Size'].apply(lambda x: float(x))

In [ ]:
#Manipulamos la columna de 'Installs'
dfApp['Installs'] = dfApp['Installs'].apply(lambda x: x.replace('+', '') if '+' in str(x) else x)
dfApp['Installs'] = dfApp['Installs'].apply(lambda x: x.replace(',', '') if ',' in str(x) else x)
dfApp['Installs'] = dfApp['Installs'].apply(lambda x: x.replace('Free', '0') if 'Free' in str(x) else x)
dfApp['Installs'] = dfApp['Installs'].apply(lambda x: int(x))
dfApp['Installs'] = dfApp['Installs'].apply(lambda x: float(x))


In [ ]:
#Manipulamos la columna de 'Price'
dfApp['Price'] = dfApp['Price'].apply(lambda x: str(x).replace('$', '') if '$' in str(x) else str(x))
dfApp['Price'] = dfApp['Price'].apply(lambda x: x.replace('Everyone', '0') if 'Everyone' in str(x) else x)
dfApp['Price'] = dfApp['Price'].apply(lambda x: float(x))

In [ ]:
#Manipulamos la columna de 'Reviews'
dfApp['Reviews'] = dfApp['Reviews'].apply(lambda x: x.replace('3.0M', '3') if '3.0M' in str(x) else x)

dfApp['Reviews'] = dfApp['Reviews'].apply(lambda x: int(x))

In [ ]:
#Comprobamos que los datos son del tipo necesario para operar con ellos
dfApp.dtypes


App                object
Category           object
Rating            float64
Reviews             int64
Size              float64
Installs          float64
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [ ]:
#Calculamos la correlación entre las columnas 'Rating' y 'Size'
corrmat = dfApp.corr()
corrmat
dfApp['Price'].corr(dfApp['Rating'])

-0.02158946997698432

In [ ]:
#Pedimos las columnas a comparar

def toInput():
    #ask for the column to compare
    while True:
        print('Elige el par de elementos que quieres comparar')
        try:
            opciones = ['Rating', 'Reviews', 'Size', 'Installs', 'Price']
            col1 = input('Choose the first: Rating, Reviews, Size, Installs oe Price: ')
            if col1 in opciones: 
                print("Correct!!")  
                
            else:
                raise ValueError ("It looks bad! Try again")

            col2 = input('Choose the second: Rating, Reviews, Size, Installs oe Price: ')   
            if col2 in opciones:
                print("Correct!!")
                print(col2)
            else:
                raise ValueError ("It looks bad! Try again") 
        
        except:
            print ("INCORRECT input")
            toInput()
        return dfApp[col1].corr(dfApp[col2])
toInput()

Elige el par de elementos que quieres comparar


In [ ]:
# Función que realiza la comparación
col1 = ''
col2 = ''

def compCorr(col1, col2):
    return dfApp[col1].corr(dfApp[col2])

print(compCorr(col1, col2))

In [ ]:
#Partíamos de la hipótesis que el Rating de las Apps estaba relacionado con su Size.
# Aplicamos la correlación entre ambas columnas y comprobamos que la hipótesis es errónea.
# Como refleja el siguiente gráfico:

#f, ax = plt.subplots()
corrmat = dfApp.corr()
p =sns.heatmap(corrmat, annot=True, cmap=sns.diverging_palette(220, 20, as_cmap=True))

In [ ]:
# Exporting DataFrame

export = dfApp.to_csv('../output/dfApp.csv', index=False)
